In [1]:
import numpy as np
from enum import Enum
import random 
import copy
import pickle 
from tqdm import tqdm_notebook
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, RNN, Conv2D, MaxPooling2D, Flatten, Dropout
import time, datetime
from IPython.display import clear_output
from keras.models import model_from_json
from uttt import UltimateTicTacToe
from helper import Turns, GameState, switch_turns, current_milli_time
import os
import keras

Using TensorFlow backend.


In [13]:
class NeuralNetwork:
    def __init__(self):
        self.model = Sequential()
        self.features = None
        self.labels = None
    
#     self.model = Sequential()
#         self.model.add(Conv2D(128, kernel_size=(3, 3),
#                          activation='relu',
#                          input_shape=(9,9,2)))
#         self.model.add(Conv2D(128, (3, 3), activation='relu'))
#         self.model.add(MaxPooling2D(pool_size=(2, 2)))
#         self.model.add(Dropout(0.2))
#         self.model.add(Flatten())
#         self.model.add(Dense(128, activation='relu'))
#         self.model.add(Dropout(0.5))
#         self.model.add(Dense(128, activation='relu'))
#         self.model.add(Dropout(0.5))
#         self.model.add(Dense(2, activation='softmax'))
    
    def csetup(self):
        self.model = Sequential()
        self.model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(18,9,1)))
        self.model.add(Conv2D(64, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(2, activation='softmax'))


        self.model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
    def setup(self):
        self.model.add(Dense(128,input_shape=(162,)))
        self.model.add(Activation('relu'))
        self.model.add(Dense(256))
        self.model.add(Activation('relu'))
        self.model.add(Dense(128))
        self.model.add(Activation('relu'))
        self.model.add(Dense(2))
        self.model.add(Activation('softmax'))

        self.model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    
    def train(self):
        if self.features is None or self.labels is None:
            print("Dude, you are missing data I think...")
            return
        self.model.fit(self.features, self.labels, epochs=10, batch_size=32)

    def set_data(self,features,labels):
        self.features = features
        self.labels = labels
    
    def predict(self,datum):
        return self.model.predict(datum)
    
    def set_model(self, model):
        self.model = model
    
    def save(self):
        timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        # saving the model
        model_json = self.model.to_json()
        filename = "models/"+timestamp
        os.makedirs(filename, exist_ok=True)
        with open("models/"+timestamp+"/model.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        self.model.save_weights("models/"+timestamp+"/weights.h5")
        print("Saved model to disk as with "+timestamp+" timestamp")
        
    def load(self):
        f = open("model.json")
        model = model_from_json(f.read())
        f.close()
        model.load_weights("weights.h5")
        print("Model loaded from the disk...")
        self.model = model
        print("Model set to the Neural Network...")
        self.model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
        print("Model compiled...")        

In [15]:
class Randomizer:
    def __init__(self, batch_size=100, q_val=1, stretch=1):
        self.batch_size = batch_size
        self.nn = NeuralNetwork()
        self.initial_board = np.array([np.zeros(9) for _ in range(9)])
        self.q_val = q_val
        self.stretch = stretch
        self.UTTT = UltimateTicTacToe(board=self.initial_board,last_turn=None)
        self.nn.setup()
        
    def run(self):
        features, labels = [], []
        for _ in tqdm_notebook(range(self.batch_size),desc="Batch Run..."):
            feature, winner = self.play_a_random_game()
            label = self.feature_and_label_of_memory(len(feature), winner)
            features.extend(feature)
            labels.extend(label)
        self.train(np.array(features), np.array(labels))
    
    def crun(self):
        features, labels = [], []
        for _ in tqdm_notebook(range(self.batch_size),desc="Batch Run..."):
            feature, winner = self.play_a_random_game()
            label = self.feature_and_label_of_memory(len(feature), winner)
            features.extend([f.reshape((18,9,1)) for f in feature])
            labels.extend(label)
        self.train(np.array(features), np.array(labels))
    
    
    def feature_and_label_of_memory(self, memory_len, winner):
        labels = []
        for i in range(memory_len):
            label = None
            value = self.q_val**(memory_len-(i+1)) * self.stretch
            if winner==Turns.X.value:
                label = np.array([value,0])
            elif winner==Turns.O.value:
                label = np.array([0,value])
            else:
                half_prob = value
                label = np.array([half_prob, half_prob])
            labels.append(label)
        return labels
                
    def play_a_random_game(self): # returns the memory of the game with the winner
            memory = []
            uttt = copy.deepcopy(self.UTTT)
            simulation_turn = Turns.X.value
            while not uttt.is_game_done():
                old_board = uttt.get_board_list().flatten()
                move = random.choice(uttt.get_free_moves())
                uttt.move(simulation_turn,*move)
                new_board = uttt.get_board_list().flatten()
                state = np.concatenate((old_board,new_board),axis=0) 
                simulation_turn = switch_turns(simulation_turn)
                memory.append(state)
            winner = uttt.get_winner() if uttt.get_winner() != None else 0
            return memory, winner
    
    def train(self, features, labels):
        self.nn.set_data(features, labels)
        self.nn.train()
#         clear_output()
#         self.nn.save()
        
    def load_last_nn_model(self):
        self.nn.load()

In [16]:
# loading the last trained model
randomizer = Randomizer(batch_size=1000)
# randomizer.load_last_nn_model()

In [17]:
now = current_milli_time()
for _ in range(1):
    randomizer.crun()
    

KeyboardInterrupt: 

In [8]:
randomizer.nn.save()

Saved model to disk as with 2018-07-27 14:45:19 timestamp
